In [ ]:
# default_exp icebox

# icebox
> Routines for working with frozen jukebox embeddings

Combination of TagBox repo by Ethan Manilow et al plus additions/modifications by Scott Hawley 

In [ ]:
#export 
from torch import nn 
from jukebox.make_models import make_vqvae, make_prior, MODELS, make_model
from jukebox.hparams import Hyperparams, setup_hparams

## TagBox utils

Utilities from Ethan Manilows's TagBox: https://github.com/ethman/tagbox, slightly modified by Scott H. Hawley @drscotthawley


In [ ]:
#export 

#JUKEBOX_SAMPLE_RATE = 44100  # ethan's original
JUKEBOX_SAMPLE_RATE = None

def init_jukebox_sample_rate(
    sr=44100  # sample rate in Hz. OpenAI's pretrained Jukebox weights are for 44100
    ): 
    "SHH added this util to preserve rest of code minimall-modified"
    global JUKEBOX_SAMPLE_RATE
    JUKEBOX_SAMPLE_RATE = sr
    return

def audio_for_jbx(audio, trunc_sec=None, device=None):
    """Readies an audio TENSOR for Jukebox."""
    if audio.ndim == 1:
        audio = audio[None]
        audio = audio.mean(axis=0)

    # normalize audio
    norm_factor = torch.abs(audio).max()
    if norm_factor > 0:
        audio /= norm_factor

    if trunc_sec is not None:  # truncate sequence
        audio = audio[: int(JUKEBOX_SAMPLE_RATE * trunc_sec)]

    audio = audio[:, :, None]  # add one more dimension on the end?
    return audio


def load_audio_for_jbx(path, offset=0.0, dur=None, trunc_sec=None, device=None):
    """Loads a path for use with Jukebox."""
    audio, sr = librosa.load(path, sr=None, offset=offset, duration=dur)

    if JUKEBOX_SAMPLE_RATE is None: init_jukebox_sample_rate()

    if sr != JUKEBOX_SAMPLE_RATE:
        audio = librosa.resample(audio, sr, JUKEBOX_SAMPLE_RATE)

    return audio_for_jbx(audio, trunc_sec, device=device)

## Icebox Encoder
frozen Jukebox encoder for embeddings

In [ ]:
#export
class IceBoxEncoder(nn.Module):
    def __init__(self, global_args, port=9500):
        super().__init__()

        n_io_channels = 2
        n_feature_channels = 8

        # for making Jukebox work with multi-GPU runs
        rank, local_rank, device = int(os.getenv('RANK')), int(os.getenv('LOCAL_RANK')), self.device
        dist_url = f"tcp://127.0.0.1:{port}"  # Note port may differ on different machines
        dist.init_process_group(backend="nccl")

        self.hps = Hyperparams()
        assert global_args.sample_rate == 44100, "Jukebox was pretrained at 44100 Hz."
        self.hps.sr = global_args.sample_rate #44100
        self.hps.levels = 3
        self.hps.hop_fraction = [.5,.5,.125]

        vqvae = "vqvae"
        self.vqvae = make_vqvae(setup_hparams(vqvae, dict(sample_length = 1048576)), self.device)
        for param in self.vqvae.parameters():  # FREEZE IT.  "IceBox"
            param.requires_grad = False

        self.encoder = self.vqvae.encode

        latent_dim = 64 # global_args.latent_dim. Jukebox is 64
        io_channels = 2#1 # 2.  Jukebox is mono but we decode in stereo
 
    def encode(self, *args, **kwargs):
        return self.encoder(*args, **kwargs)